## Event reproducibility in ALM
### data from Li et al. 2015
See the paper:    
*"A motor cortex circuit for motor planning and movement"*    
Nuo Li, Tsai-Wen Chen, Zengcai V. Guo, Charles R. Gerfen & Karel Svoboda    
Nature 519, 51–56 (05 March 2015) doi: 10.1038/nature14178

In [4]:
from platform import python_version
print(python_version())

%run -i 'imports_functions.py' 

%matplotlib inline

3.10.4


### Data preprocessing

The original files `alm-2` containing the DF/F traces is available on [CRCNS](http://crcns.org/data-sets/motor-cortex/alm-2).

Here, to speed up the processing and reduce the dependencies, we use the Ca spiketrains extracted using the `deconvolve` function available in the library [OASIS](https://github.com/j-friedrich/OASIS/pull/19) (with all default parameters, and `penalty` of 1).

In [5]:
exp_path = os.getcwd() + "/svoboda/"

frame_duration = 0.1 # sec, 10 frames per second, see Methods in Franco and Goard 2021

We use the separate 57 sessions independently

In [6]:
ophys_experiments = [
    # animal 19
    "data_an019_2013_08_21_210",
    "data_an019_2013_08_20_245",
    "data_an019_2013_08_20_275",
    "data_an019_2013_08_21_240",
    "data_an019_2013_08_21_270",
    "data_an019_2013_08_16_280",
    "data_an019_2013_08_16_310",
    "data_an019_2013_08_16_340",
    "data_an019_2013_08_16_370",
    "data_an019_2013_08_19_410",
    "data_an019_2013_08_19_440",
    "data_an019_2013_08_19_470",
    "data_an019_2013_08_15_480",
    "data_an019_2013_08_19_500",
    "data_an019_2013_08_15_510",
    "data_an019_2013_08_19_530",
    "data_an019_2013_08_15_570",
    "data_an019_2013_08_21_600",
    "data_an019_2013_08_21_630",
    # animal 22
    "data_an022_2013_08_26_470",
    "data_an022_2013_08_26_500",
    "data_an022_2013_08_26_530",
    "data_an022_2013_08_22_550",
    "data_an022_2013_08_26_560",
    "data_an022_2013_08_23_580",
    "data_an022_2013_08_23_610",
    "data_an022_2013_08_23_640",
    "data_an022_2013_08_25_620",
    "data_an022_2013_08_25_650",
    "data_an022_2013_08_25_680",
    "data_an022_2013_08_25_710",
    "data_an022_2013_08_25_740",
    # animal 23
    "data_an023_2013_10_14_210",
    "data_an023_2013_10_14_240",
    "data_an023_2013_10_15_300",
    "data_an023_2013_10_15_330",
    "data_an023_2013_10_15_360",
    "data_an023_2013_10_10_480",
    "data_an023_2013_10_10_505",
    "data_an023_2013_10_09_530",
    "data_an023_2013_10_11_530",
    "data_an023_2013_10_10_535",
    "data_an023_2013_10_11_560",
    # animal 26
    "data_an026_2013_11_10_150",
    "data_an026_2013_11_10_180",
    "data_an026_2013_11_10_210",
    "data_an026_2013_11_10_240",
    "data_an026_2013_11_10_270",
    "data_an026_2013_11_12_485",
    "data_an026_2013_11_08_505",
    "data_an026_2013_11_12_515",
    "data_an026_2013_11_08_535",
    "data_an026_2013_11_12_545",
    "data_an026_2013_11_14_550",
    "data_an026_2013_11_08_560",
    "data_an026_2013_11_12_575",
    "data_an026_2013_11_14_580",
]

## Dynamical analysis

Analysis for each session.

On average, the longest responses were for the 4th trial. We take it.

In [ ]:
global_events = [[] for i in range(57)] # by session
global_events_vector = [[] for i in range(57)] # by session
global_events_sec = []
global_events_duration = []
global_cluster_number = []
global_cluster_selfsimilarity = [[] for i in range(57)]

core_reproducibility_perc = 60 # threshold for detecting cores

sessions_evt_toBremoved = [[] for i in range(57)] # by session
sessions_evt_toBremoved[1] = [0] 

for scan_id,oe in enumerate(ophys_experiments):
    # pprint.pprint(oe)

    print("\n\n%d/%d" % (scan_id+1,len(ophys_experiments)))

    if os.path.exists(exp_path+'/spiketrains_'+str(oe)+'.npy') and os.path.exists(exp_path+'/time_'+str(oe)+'.npy'):
        scan_spiketrains = np.load(exp_path+'/spiketrains_'+str(oe)+'.npy', allow_pickle=True)
        time = np.load(exp_path+'/time_'+str(oe)+'.npy', allow_pickle=True)
        print("... loaded",len(scan_spiketrains), "spiketrains")
        # print(time)

    print(len(scan_spiketrains))
    
    start_time = 0
    exp_tstart = start_time
    stop_time = max([max(st) if len(st) else 0 for st in scan_spiketrains])
    
    print("... producing spike rasterplot per session")
    fig = plt.figure()
    for row,train in enumerate(scan_spiketrains):
        plt.scatter( train, [row]*len(train), marker='o', edgecolors='none', s=1, c='k' )
    plt.ylabel("cell IDs")
    plt.xlabel("time (s)")
    fig.savefig(exp_path+'/results/rasterplot_scan%s.png'%scan_id, transparent=True, dpi=800)
    plt.close()
    fig.clear()
    fig.clf()
    
    ophys_cell_ids = list(range(len(scan_spiketrains)))
    ophys_cell_indexes = ophys_cell_ids # alias
    
    %run "dynamical_analysis.ipynb"

    global_events[scan_id].extend(events)
    global_events_vector[scan_id].extend(events_vectors)
    global_events_sec.append(events_sec)
    global_events_duration.extend(events_durations_f)
    global_cluster_number.append(nclusters)
    global_cluster_selfsimilarity.extend(reproducibility_list)




1/57
... loaded 49 spiketrains
49
... producing spike rasterplot per session
    population firing: 6.11±4.16 sp/frame
    cells firing rate: 0.12±0.37 sp/s
... generating surrogates to establish population event threshold
... loaded surrogates
    event size threshold (mean): 17.44482723166876
... find peaks
... find minima
... find population events
... signatures of population events
    number of events: 7
    number of events per sec: 0.01362539349422875
    events duration: 1.200±0.512
    events size: 35.000±2.119
... Similarity of events matrix
... clustering
    linkage
    surrogate events signatures for clustering threshold
... loaded surrogates
    cluster reproducibility threshold: 0.0
    cluster size threshold: 2
    Total number of clusters: 4
    # clusters (after removing those below reproducibility threshold): 2
... finding cluster cores
    removing cores firing unspecifically
    gathering cores from all clusters
    # cores: 43
    # non-cores: 6
    cores per c